In [1]:
# importing relevant packages 

import torch 
from torchvision import models
import pandas as pd
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn 
import glob
import os
import cv2
import matplotlib.pyplot as plt 
from matplotlib import pyplot
import torchvision.transforms as transforms
from torchsummary import summary
from sklearn.model_selection import train_test_split
import numpy as np
from tqdm import tqdm
import json
from PIL import Image
import random
import albumentations as A
import glob
from shapely.geometry import Polygon

/home/dell/Desktop/xxxx/Uni/Kirigami_project/Keypoint_detection_notebooks/.Keypoint_detection_notebooks/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.14) or chardet (5.1.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [2]:
class VGG_model(nn.Module):
    def __init__(self):
        super(VGG_model, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.LeakyReLU(0.1,inplace=True),
            nn.Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.LeakyReLU(0.1,inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False),
            nn.Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.LeakyReLU(0.1,inplace=True),
            nn.Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.LeakyReLU(0.1,inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False),
            nn.Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.LeakyReLU(0.1,inplace=True),
            nn.Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.LeakyReLU(0.1,inplace=True),
            nn.Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.LeakyReLU(0.1,inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False),
            nn.Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.LeakyReLU(0.1,inplace=True),
            nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.LeakyReLU(0.1,inplace=True),
            nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.LeakyReLU(0.1,inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False),
            nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.LeakyReLU(0.1,inplace=True),
            nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.LeakyReLU(0.1,inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
        )
        
        self.avgpool = nn.Sequential(
            nn.Conv2d(512,512, kernel_size=3, padding='same'),
            nn.LeakyReLU(0.1,inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(512,50, kernel_size=3, padding='same'),
            nn.LeakyReLU(0.1,inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.AdaptiveAvgPool2d(output_size=(8,8))
        )
        
        self.classifier = nn.Sequential(
            nn.Linear(3200, 300),
            nn.LeakyReLU(0.1,inplace=True),
            nn.Dropout(0.3),
            nn.Linear(300, 32),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x
        

In [3]:
model = VGG_model()
model.load_state_dict(torch.load('./Model_VGG_7_1000_22100_epoch'))
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = model.to(device)
model.eval()

VGG_model(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): LeakyReLU(negative_slope=0.1, inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): LeakyReLU(negative_slope=0.1, inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): LeakyReLU(negative_slope=0.1, inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): LeakyReLU(negative_slope=0.1, inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): LeakyReLU(negative_slope=0.1, inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): LeakyReLU(negative_slope=0.1, inplace=True)
    (14): Conv2d(

In [4]:
def predict_keypoints(img):
    img = cv2.resize(img, (224,224))
    img_tensor = transforms.ToTensor()(img)
    img_mean = img_tensor.mean(dim = (1,2))
    img_std = img_tensor.std(dim = (1,2))
        
    img_normalised = transforms.Normalize(img_mean, img_std)(img_tensor)
    img_normalised = img_normalised.to(device)
    
    key_points = model(img_normalised[None]).flatten().detach().cpu().numpy()
    
    return key_points

In [5]:
def plot_keypoints(img, keypoints):                                                             

    plt.imshow(img)

    x_points = keypoints[0::2]
    y_points = keypoints[1::2]
    plt.scatter(x_points*img.shape[1], y_points*img.shape[0], s = 4, c=(1,0,0))
    plt.show()

In [6]:
def process_num(x):
    return x*random.uniform(0.96, 1.04)

In [24]:
dataset_names = ['Grasp_dataset_eyedropper', 'Grasp_dataset_testtube', 'Grasp_dataset_squarebottle', 'Grasp_dataset_roundbottlesmall', 'Grasp_dataset_roundbottlemedium', 'Grasp_dataset_roundbottlelarge']

header = ['weight_reading_1', 'weight_reading_2', 'pressure_reading_1', 'pressure_reading_2', 'force_reading_1', 'force_reading_2', 'p1_x', 'p1_y', 'p2_x', 'p2_y', 'p3_x', 'p3_y', 'p4_x', 'p4_y', 'p5_x', 'p5_y', 'p6_x', 'p6_y', 'p7_x', 'p7_y', 'p8_x', 'p8_y', 'p9_x', 'p9_y', 'p10_x', 'p10_y', 'p11_x', 'p11_y', 'p12_x', 'p12_y', 'p13_x', 'p13_y', 'p14_x', 'p14_y', 'p15_x', 'p15_y', 'p16_x', 'p16_y', 'label']

current_dir = os. getcwd()

weight_reading_1_arr = []
weight_reading_2_arr = []
pressure_reading_1_arr = []
pressure_reading_2_arr = []
force_reading_1_arr = []
force_reading_2_arr = []

p1_x_arr = []
p1_y_arr = []
p2_x_arr = []
p2_y_arr = []
p3_x_arr = []
p3_y_arr = []
p4_x_arr = []
p4_y_arr = []
p5_x_arr = []
p5_y_arr = []
p6_x_arr = []
p6_y_arr = []
p7_x_arr = []
p7_y_arr = []
p8_x_arr = []
p8_y_arr = []
p9_x_arr = []
p9_y_arr = []
p10_x_arr = []
p10_y_arr = []
p11_x_arr = []
p11_y_arr = []
p12_x_arr = []
p12_y_arr = []
p13_x_arr = []
p13_y_arr = []
p14_x_arr = []
p14_y_arr = []
p15_x_arr = []
p15_y_arr = []
p16_x_arr = []
p16_y_arr = []
label_arr = []


for f in dataset_names:
    
#     image_dir = os.path.join(current_dir + '/Grasp_dataset_7', f)
    image_dir = os.path.join(current_dir + '/Grasp_dataset_7_test', f)
    data_dir = os.path.join(image_dir, f + '.csv')
    df = pd.read_csv(data_dir)
    weight_reading_1_arr = np.concatenate((weight_reading_1_arr, df['weight_reading_1'].values))
    weight_reading_2_arr = np.concatenate((weight_reading_2_arr, df['weight_reading_2'].values))
    pressure_reading_1_arr = np.concatenate((pressure_reading_1_arr, df['pressure_reading_1'].values))
    pressure_reading_2_arr = np.concatenate((pressure_reading_2_arr, df['pressure_reading_2'].values))
    force_reading_1_arr = np.concatenate((force_reading_1_arr, df['force_reading_1'].values))
    force_reading_2_arr = np.concatenate((force_reading_2_arr, df['force_reading_2'].values))
    label_arr = np.concatenate((label_arr, df['label'].values))
    
    j = 0
    
    while j < df.shape[0]:
        image = cv2.imread(os.path.join(image_dir, df.iloc[j]['image_name']))
        keypoints = predict_keypoints(image)
        
#         plot_keypoints(image, keypoints)
        
#         if j == 20:
#             plot_keypoints(image, keypoints)
        

#         plt.imshow(image)

#         plt.scatter(kirigami_x_points*image.shape[1], kirigami_y_points*image.shape[0], s = 4, c=(1,0,0))
#         plt.show()
        
        p1_x_arr.append(keypoints[0])
        p1_y_arr.append(keypoints[1])
        p2_x_arr.append(keypoints[2])
        p2_y_arr.append(keypoints[3])
        p3_x_arr.append(keypoints[4])
        p3_y_arr.append(keypoints[5])
        p4_x_arr.append(keypoints[6])
        p4_y_arr.append(keypoints[7])
        p5_x_arr.append(keypoints[8])
        p5_y_arr.append(keypoints[9])
        p6_x_arr.append(keypoints[10])
        p6_y_arr.append(keypoints[11])
        p7_x_arr.append(keypoints[12])
        p7_y_arr.append(keypoints[13])
        p8_x_arr.append(keypoints[14])
        p8_y_arr.append(keypoints[15])
        p9_x_arr.append(keypoints[16])
        p9_y_arr.append(keypoints[17])
        p10_x_arr.append(keypoints[18])
        p10_y_arr.append(keypoints[19])
        p11_x_arr.append(keypoints[20])
        p11_y_arr.append(keypoints[21])
        p12_x_arr.append(keypoints[22])
        p12_y_arr.append(keypoints[23])
        p13_x_arr.append(keypoints[24])
        p13_y_arr.append(keypoints[25])
        p14_x_arr.append(keypoints[26])
        p14_y_arr.append(keypoints[27])
        p15_x_arr.append(keypoints[28])
        p15_y_arr.append(keypoints[29])
        p16_x_arr.append(keypoints[30])
        p16_y_arr.append(keypoints[31])
        
        j = j + 1
        
#         if j == 2:
#             break
#     break



p1_x_arr = np.array(p1_x_arr) - np.array(p1_x_arr)
p2_x_arr = np.array(p2_x_arr) - np.array(p1_x_arr)
p3_x_arr = np.array(p3_x_arr) - np.array(p1_x_arr)
p4_x_arr = np.array(p4_x_arr) - np.array(p1_x_arr)
p5_x_arr = np.array(p5_x_arr) - np.array(p1_x_arr)
p6_x_arr = np.array(p6_x_arr) - np.array(p1_x_arr)
p7_x_arr = np.array(p7_x_arr) - np.array(p1_x_arr)
p8_x_arr = np.array(p8_x_arr) - np.array(p1_x_arr)
p9_x_arr = np.array(p9_x_arr) - np.array(p1_x_arr)
p10_x_arr = np.array(p10_x_arr) - np.array(p1_x_arr)
p11_x_arr = np.array(p11_x_arr) - np.array(p1_x_arr)
p12_x_arr = np.array(p12_x_arr) - np.array(p1_x_arr)
p13_x_arr = np.array(p13_x_arr) - np.array(p1_x_arr)
p14_x_arr = np.array(p14_x_arr) - np.array(p1_x_arr)
p15_x_arr = np.array(p15_x_arr) - np.array(p1_x_arr)
p16_x_arr = np.array(p16_x_arr) - np.array(p1_x_arr)

p1_y_arr = np.array(p1_y_arr) - np.array(p1_y_arr)
p2_y_arr = np.array(p2_y_arr) - np.array(p1_y_arr)
p3_y_arr = np.array(p3_y_arr) - np.array(p1_y_arr)
p4_y_arr = np.array(p4_y_arr) - np.array(p1_y_arr)
p5_y_arr = np.array(p5_y_arr) - np.array(p1_y_arr)
p6_y_arr = np.array(p6_y_arr) - np.array(p1_y_arr)
p7_y_arr = np.array(p7_y_arr) - np.array(p1_y_arr)
p8_y_arr = np.array(p8_y_arr) - np.array(p1_y_arr)
p9_y_arr = np.array(p9_y_arr) - np.array(p1_y_arr)
p10_y_arr = np.array(p10_y_arr) - np.array(p1_y_arr)
p11_y_arr = np.array(p11_y_arr) - np.array(p1_y_arr)
p12_y_arr = np.array(p12_y_arr) - np.array(p1_y_arr)
p13_y_arr = np.array(p13_y_arr) - np.array(p1_y_arr)
p14_y_arr = np.array(p14_y_arr) - np.array(p1_y_arr)
p15_y_arr = np.array(p15_y_arr) - np.array(p1_y_arr)
p16_y_arr = np.array(p16_y_arr) - np.array(p1_y_arr)


weight_diff_1_arr = np.array(weight_reading_1_arr) - np.array(force_reading_1_arr)
weight_diff_2_arr = np.array(force_reading_2_arr) - np.array(weight_reading_2_arr) 

grasp_dataset = pd.concat([pd.Series(weight_diff_1_arr), pd.Series(weight_diff_2_arr), pd.Series(pressure_reading_1_arr), pd.Series(pressure_reading_2_arr), pd.Series(force_reading_1_arr), pd.Series(force_reading_2_arr), pd.Series(p1_x_arr), pd.Series(p1_y_arr), pd.Series(p2_x_arr), pd.Series(p2_y_arr), pd.Series(p3_x_arr), pd.Series(p3_y_arr), pd.Series(p4_x_arr), pd.Series(p4_y_arr), pd.Series(p5_x_arr), pd.Series(p5_y_arr), pd.Series(p6_x_arr), pd.Series(p6_y_arr), pd.Series(p7_x_arr), pd.Series(p7_y_arr), pd.Series(p8_x_arr), pd.Series(p8_y_arr), pd.Series(p9_x_arr), pd.Series(p9_y_arr), pd.Series(p10_x_arr), pd.Series(p10_y_arr), pd.Series(p11_x_arr), pd.Series(p11_y_arr), pd.Series(p12_x_arr), pd.Series(p12_y_arr), pd.Series(p13_x_arr), pd.Series(p13_y_arr), pd.Series(p14_x_arr), pd.Series(p14_y_arr), pd.Series(p15_x_arr), pd.Series(p15_y_arr), pd.Series(p16_x_arr), pd.Series(p16_y_arr), pd.Series(label_arr)], axis=1, keys=header)

# num_cols = grasp_dataset.select_dtypes(include=['float']).columns
# df[num_cols] = grasp_dataset[num_cols].applymap(process_num)

# combined_df_set = grasp_dataset
# combined_df_set = pd.concat([combined_df_set, grasp_dataset], axis=0)


In [25]:
pd.options.display.max_columns = None
print(grasp_dataset.head(3))

   weight_reading_1  weight_reading_2  pressure_reading_1  pressure_reading_2  \
0              65.8              62.1                51.7                54.9   
1              67.8              60.1                47.4                56.7   
2              67.1              60.5                44.5                53.2   

   force_reading_1  force_reading_2  p1_x  p1_y      p2_x      p2_y      p3_x  \
0            258.9            258.2   0.0   0.0  0.477672  0.178696  0.796955   
1            261.8            260.9   0.0   0.0  0.478581  0.173928  0.797436   
2            247.3            246.2   0.0   0.0  0.476830  0.185747  0.796214   

       p3_y      p4_x      p4_y      p5_x      p5_y      p6_x      p6_y  \
0  0.358088  0.795623  0.699320  0.484259  0.857428  0.181057  0.712818   
1  0.358294  0.795502  0.697292  0.483962  0.861309  0.179839  0.711746   
2  0.354417  0.795066  0.700851  0.483582  0.853393  0.181343  0.712325   

       p7_x      p7_y      p8_x      p8_y      p9

In [26]:
# pd.set_option('display.max_rows', 500)
grasp_dataset

,weight_reading_1,weight_reading_2,pressure_reading_1,pressure_reading_2,force_reading_1,force_reading_2,p1_x,p1_y,p2_x,p2_y,p3_x,p3_y,p4_x,p4_y,p5_x,p5_y,p6_x,p6_y,p7_x,p7_y,p8_x,p8_y,p9_x,p9_y,p10_x,p10_y,p11_x,p11_y,p12_x,p12_y,p13_x,p13_y,p14_x,p14_y,p15_x,p15_y,p16_x,p16_y,label
0,65.8,62.1,51.7,54.9,258.9,258.2,0.0,0.0,0.477672,0.178696,0.796955,0.358088,0.795623,0.699320,0.484259,0.857428,0.181057,0.712818,0.463786,0.259964,0.459817,0.387287,0.461804,0.526016,0.467785,0.656556,0.472408,0.777460,0.498506,0.777903,0.504491,0.655084,0.506821,0.523901,0.501221,0.397237,0.495070,0.267498,eyedropper
1,67.8,60.1,47.4,56.7,261.8,260.9,0.0,0.0,0.478581,0.173928,0.797436,0.358294,0.795502,0.697292,0.483962,0.861309,0.179839,0.711746,0.464104,0.259620,0.458867,0.387484,0.460199,0.525493,0.466351,0.655193,0.471920,0.776751,0.497826,0.777362,0.504764,0.653930,0.507641,0.523071,0.502019,0.396902,0.495270,0.266812,eyedropper
2,67.1,60.5,44.5,53.2,247.3,246.2,0.0,0.0,0.476830,0.185747,0.796214,0.354417,0.795066,0.700851,0.483582,0.853393,0.181343,0.712325,0.463460,0.260338,0.460560,0.386782,0.462960,0.525885,0.468647,0.656994,0.472224,0.777353,0.498123,0.777590,0.503344,0.655353,0.505433,0.523953,0.500112,0.396889,0.494563,0.267884,eyedropper
3,66.8,61.0,43.2,49.9,249.8,248.8,0.0,0.0,0.477455,0.184569,0.796176,0.356358,0.795049,0.699430,0.483648,0.856178,0.180473,0.713080,0.463485,0.260972,0.459907,0.387560,0.461896,0.526261,0.467666,0.657048,0.471876,0.777786,0.497846,0.777942,0.503398,0.655378,0.505713,0.524138,0.500329,0.397393,0.494573,0.268287,eyedropper
4,67.8,60.1,45.6,49.4,242.8,241.7,0.0,0.0,0.477144,0.188543,0.795785,0.356346,0.794800,0.703070,0.483543,0.850099,0.181721,0.713191,0.463605,0.260629,0.461102,0.386690,0.463577,0.525956,0.469113,0.657279,0.472321,0.777170,0.498054,0.777326,0.502867,0.655651,0.504860,0.524321,0.499711,0.397109,0.494610,0.268323,eyedropper
5,66.7,60.8,42.8,52.9,249.6,248.2,0.0,0.0,0.477740,0.186209,0.796343,0.357161,0.794929,0.700568,0.483251,0.856215,0.180570,0.712072,0.463716,0.261443,0.460193,0.387845,0.462171,0.526646,0.467811,0.657657,0.471771,0.778282,0.497711,0.778417,0.503208,0.655964,0.505630,0.524776,0.500368,0.398050,0.494851,0.268966,eyedropper
6,67.5,60.3,50.8,64.8,264.3,263.4,0.0,0.0,0.479220,0.172729,0.798229,0.359583,0.794774,0.698670,0.482950,0.863306,0.179173,0.711192,0.464618,0.260356,0.458714,0.388489,0.459585,0.526316,0.465516,0.655788,0.471228,0.777306,0.497086,0.777633,0.504809,0.654105,0.508303,0.523752,0.502941,0.398073,0.496040,0.267577,eyedropper
7,67.7,59.8,52.0,62.6,269.1,268.0,0.0,0.0,0.479073,0.169607,0.799166,0.357538,0.795231,0.701586,0.482074,0.865104,0.180278,0.708994,0.465271,0.259307,0.459351,0.387938,0.460293,0.526218,0.466141,0.655989,0.471690,0.777645,0.497287,0.777826,0.505970,0.653850,0.509885,0.523681,0.504375,0.398343,0.496830,0.267083,eyedropper
8,67.2,61.0,52.1,64.7,264.5,263.8,0.0,0.0,0.479535,0.171125,0.798813,0.357280,0.793883,0.695431,0.482770,0.862395,0.178386,0.714722,0.464994,0.260091,0.458717,0.388537,0.459283,0.526422,0.464944,0.655753,0.470732,0.777076,0.496402,0.776775,0.504356,0.653207,0.508161,0.522942,0.503143,0.397382,0.496410,0.266691,eyedropper
9,68.5,59.3,51.2,64.8,264.5,263.6,0.0,0.0,0.477686,0.172362,0.799360,0.354650,0.795943,0.706028,0.481128,0.864372,0.180932,0.706309,0.465036,0.259587,0.460093,0.387952,0.461454,0.526708,0.467173,0.656960,0.471918,0.778511,0.497462,0.778706,0.506088,0.654829,0.509841,0.524439,0.504207,0.398759,0.496499,0.267579,eyedropper


In [16]:
# grasp_dataset.to_csv('/home/dell/Desktop/xxxx/Uni/Kirigami_project/Keypoint_detection_notebooks/Grasp_dataset_3/' + 'validation_df.csv', index=False)

In [27]:
# grasp_data = grasp_dataset.drop(columns=['label', 'weight_reading_1', 'weight_reading_2'], axis=1)
grasp_data = grasp_dataset.drop(columns=['label'], axis=1)
# grasp_data = grasp_dataset.drop(columns=['label', 'p1_x', 'p1_y', 'p2_x', 'p2_y', 'p3_x', 'p3_y', 'p4_x', 'p4_y', 'p5_x', 'p5_y', 'p6_x', 'p6_y', 'p7_x', 'p7_y', 'p8_x', 'p8_y', 'p9_x', 'p9_y', 'p10_x', 'p10_y', 'p11_x', 'p11_y', 'p12_x', 'p12_y', 'p13_x', 'p13_y', 'p14_x', 'p14_y', 'p15_x', 'p15_y', 'p16_x', 'p16_y'], axis=1)
grasp_data['weight_reading_1'] = grasp_data['weight_reading_1'] / 1000
grasp_data['weight_reading_2'] = grasp_data['weight_reading_2'] / 1000
grasp_data['pressure_reading_1'] = grasp_data['pressure_reading_1'] / 100
grasp_data['pressure_reading_2'] = grasp_data['pressure_reading_2'] / 100
grasp_data['force_reading_1'] = grasp_data['force_reading_1'] / 1000
grasp_data['force_reading_2'] = grasp_data['force_reading_2'] / 1000
grasp_label = grasp_dataset['label']

print(grasp_data.head(2))
print(grasp_label.head(2))

   weight_reading_1  weight_reading_2  pressure_reading_1  pressure_reading_2  \
0            0.0658            0.0621               0.517               0.549   
1            0.0678            0.0601               0.474               0.567   

   force_reading_1  force_reading_2  p1_x  p1_y      p2_x      p2_y      p3_x  \
0           0.2589           0.2582   0.0   0.0  0.477672  0.178696  0.796955   
1           0.2618           0.2609   0.0   0.0  0.478581  0.173928  0.797436   

       p3_y      p4_x      p4_y      p5_x      p5_y      p6_x      p6_y  \
0  0.358088  0.795623  0.699320  0.484259  0.857428  0.181057  0.712818   
1  0.358294  0.795502  0.697292  0.483962  0.861309  0.179839  0.711746   

       p7_x      p7_y      p8_x      p8_y      p9_x      p9_y     p10_x  \
0  0.463786  0.259964  0.459817  0.387287  0.461804  0.526016  0.467785   
1  0.464104  0.259620  0.458867  0.387484  0.460199  0.525493  0.466351   

      p10_y     p11_x     p11_y     p12_x     p12_y     p13_

In [23]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix
from sklearn import svm
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn import tree
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier

data_train, data_test, label_train, label_test = train_test_split(grasp_data, grasp_label, test_size=0.2, random_state=np.random.randint(100))

# print(X_train.head(3))

# clf = KNeighborsClassifier(n_neighbors=7)

# knn.fit(X_train, y_train)

# y_pred = knn.predict(X_test)

# accuracy = knn.score(X_test, y_test)
# print('Accuracy:', accuracy)
# clf = RandomForestClassifier(n_estimators=1000, max_depth=50, random_state=np.random.randint(100))
clf = RandomForestClassifier(n_estimators=1000, max_depth=20, random_state=np.random.randint(100), n_jobs = -1)
# clf = HistGradientBoostingClassifier(max_iter=200, random_state=np.random.randint(100))
# clf = GradientBoostingClassifier(n_estimators=200, learning_rate=0.1, max_depth=1, random_state=0)
# clf = VotingClassifier(estimators=[('lr', clf1), ('rf', clf2), ('gnb', clf3)],voting='hard')

# num_folds = 10
# cv_method = KFold(n_splits=num_folds, shuffle=True, random_state=np.random.randint(100))
# cv_results = cross_val_score(clf, grasp_data, grasp_label, cv=cv_method, scoring='accuracy')

# print('Cross-validation results:', cv_results)
# print('Average accuracy:', cv_results.mean())

clf.fit(data_train, label_train)
label_pred = clf.predict(data_test)
cm = confusion_matrix(label_test, label_pred)

# short_dataset_names = ['chicken', 'tomato']


# cm_df = pd.DataFrame(cm, index=short_dataset_names, columns=short_dataset_names)

# print(cm_df)
# print(cm)

accuracy = clf.score(data_test, label_test)
print("Accuracy:", accuracy)

Accuracy: 0.9583333333333334


In [28]:
label_pred = clf.predict(grasp_data)
print(label_pred)
accuracy = clf.score(grasp_data, grasp_label)
print("Accuracy:", accuracy)

['eyedropper' 'eyedropper' 'eyedropper' 'eyedropper' 'eyedropper'
 'eyedropper' 'eyedropper' 'eyedropper' 'eyedropper' 'eyedropper'
 'testtube' 'testtube' 'testtube' 'testtube' 'testtube' 'testtube'
 'testtube' 'testtube' 'testtube' 'testtube' 'squarebottle' 'squarebottle'
 'squarebottle' 'squarebottle' 'squarebottle' 'squarebottle'
 'squarebottle' 'squarebottle' 'squarebottle' 'squarebottle'
 'roundbottlesmall' 'roundbottlesmall' 'roundbottlesmall'
 'roundbottlesmall' 'roundbottlesmall' 'roundbottlesmall'
 'roundbottlesmall' 'roundbottlesmall' 'roundbottlesmall'
 'roundbottlesmall' 'roundbottlemedium' 'roundbottlemedium'
 'roundbottlemedium' 'roundbottlemedium' 'roundbottlemedium'
 'roundbottlemedium' 'roundbottlemedium' 'roundbottlemedium'
 'roundbottlemedium' 'roundbottlemedium' 'roundbottlelarge'
 'roundbottlelarge' 'roundbottlelarge' 'roundbottlelarge'
 'roundbottlelarge' 'roundbottlelarge' 'roundbottlelarge'
 'roundbottlelarge' 'squarebottle' 'roundbottlelarge']
Accuracy: 0.98333